# 2.1 Introducing Scala the language: an example

In [2]:
// A commment!
/* Another comment */
/** A documentation comment */
object MyModule {
    def abs(n: Int): Int = 
        if (n < 0) -n
        else n
    
    // private
    def formatAbs(x: Int) = {
        val msg = "The absolute value of %d is %d"
        msg.format(x, abs(x))
    }
}

defined object MyModule

In [3]:
import MyModule._
formatAbs(-42)

import MyModule._

res2_1: String = "The absolute value of -42 is 42"

# 2.2 Running our program

In [5]:
import $ivy.`com.lihaoyi::os-lib:0.9.1`
println(os.read(os.pwd / "ch02" / "MyModule.scala"))

// A commment!
/* Another comment */
/** A documentation comment */
object MyModule {
    def abs(n: Int): Int = 
        if (n < 0) -n
        else n
    
    private def formatAbs(x: Int) = {
        val msg = "The absolute value of %d is %d"
        msg.format(x, abs(x))
    }
    
    def main(args: Array[String]): Unit =
        println(formatAbs(-42))
}


import $ivy.$                          


In [6]:
os.proc("scala", "ch02/MyModule.scala").call().out.text

res5: String = """The absolute value of -42 is 42
"""

# 2.3 Modules, objects, and namespaces

In [7]:
import MyModule.abs
abs(-42)

import MyModule.abs

res6_1: Int = 42

In [8]:
import MyModule._

import MyModule._

# 2.4 Higher-order functions: passing functions to functions

In [9]:
def factorial(n: Int): Int = {
    @annotation.tailrec
    def go(n: Int, acc: Int): Int = {
        if (n <= 0) acc
        else go(n-1, n*acc)
    }
    
    go(n, 1)
}

defined function factorial

In [10]:
factorial(6)

res9: Int = 720

## EXERCISE 2.1

Write a recursive function to get the nth Fibonacci number (http://mng.bz/C29s).
The first two Fibonacci numbers are 0 and 1. The nth number is always the sum of the
previous two—the sequence begins 0, 1, 1, 2, 3, 5. Your definition should use a
local tail-recursive function.

```scala
def fib(n: Int): Int
```

In [12]:
def fib0(n: Int): Int = {
    if (n < 2) n
    else fib0(n-2) + fib0(n-1)
}

println(
    (for {
        i <- 0 to 10
    } yield fib0(i)).mkString(", "))

0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55


defined function fib0

In [13]:
def fib(n: Int): Int = {
    @annotation.tailrec
    def go(p: (Int, Int), idx: Int): (Int, Int) = {
        if (idx == 0) p
        else go((p._2, p._1 + p._2), idx-1)
    }
    
    go((0,1), n)._1
}

println(
    (for {
        i <- 0 to 10
    } yield fib(i)).mkString(", "))


0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55


defined function fib

In [14]:
object MyModule {
    def abs(n: Int): Int = 
        if (n < 0) -n
        else n
    
    // private
    def formatAbs(x: Int) = {
        val msg = "The absolute value of %d is %d"
        msg.format(x, abs(x))
    }
    
    // private
    def formatFactorial(n: Int) = {
        val msg = "The factorial of %d is %d."
        msg.format(n, factorial(n))
    }
}

// import MyModule._
MyModule.formatAbs(-42)
MyModule.formatFactorial(7)

defined object MyModule
res13_1: String = "The absolute value of -42 is 42"
res13_2: String = "The factorial of 7 is 5040."

In [15]:
def formatResult(name: String, n: Int, f: Int => Int) = {
    val msg = "The %s of %d is %d."
    msg.format(name, n, f(n))
}

formatResult("absolute value", -42, abs)
formatResult("factorial", 7, factorial)

defined function formatResult
res14_1: String = "The absolute value of -42 is 42."
res14_2: String = "The factorial of 7 is 5040."

# 2.5 Polymorphic functions: abstracting over types

In [16]:
def findFirst(ss: Array[String], key: String): Int = {
    @annotation.tailrec
    def loop(n: Int): Int = 
        if (n >= ss.length) -1
        else if (ss(n) == key) n
        else loop(n + 1)
    
    loop(0)
}

defined function findFirst

In [17]:
def findFirst[A](as: Array[A], p: A => Boolean): Int = {
    @annotation.tailrec
    def loop(n: Int): Int = 
        if (n >= as.length) -1
        else if (p(as(n))) n
        else loop(n + 1)
    
    loop(0)
}

defined function findFirst

## EXERCISE 2.2

Implement isSorted, which checks whether an Array[A] is sorted according to a
given comparison function:

```scala
def isSorted[A](as: Array[A], ordered: (A,A) => Boolean): Boolean
```

In [20]:
def isSorted[A](as: Array[A], ordered: (A,A) => Boolean): Boolean = {
    def loop(n: Int): Boolean =
        if (n >= as.length) true
        else if (ordered(as(n), as(n-1))) false
        else loop(n + 1)
    if (as.length == 0) true
    else loop(1)
}

val asc: (Int, Int) => Boolean = _ <= _
isSorted(Array(1), asc)
isSorted(Array(1, 2), asc)
isSorted(Array(2, 1), asc)

defined function isSorted
asc: (Int, Int) => Boolean = ammonite.$sess.cmd19$Helper$$Lambda$2776/1046247325@38665f40
res19_2: Boolean = true
res19_3: Boolean = true
res19_4: Boolean = false

In [21]:
findFirst(Array(7, 9, 13), (x: Int) => x == 9)

res20: Int = 1

In [22]:
(x: Int, y: Int) => x == y

res21: (Int, Int) => Boolean = ammonite.$sess.cmd21$Helper$$Lambda$2783/780474893@12b4e4fd

In [23]:
val lessThan = new Function2[Int, Int, Boolean] {
    def apply(a: Int, b: Int) = a < b
}
val b = lessThan(10, 20)

lessThan: AnyRef with (Int, Int) => Boolean = <function2>
b: Boolean = true

# 2.6 Following types to implementations

In [24]:
def partial1[A,B,C](a: A, f: (A, B) => C): B => C =
    (b: B) => f(a, b)

defined function partial1

## EXERCISE 2.3

Let’s look at another example, currying, which converts a function f of two arguments
into a function of one argument that partially applies f. Here again there’s only one
implementation that compiles. Write this implementation.

```scala
def curry[A,B,C](f: (A, B) => C): A => (B => C)
```

In [25]:
def curry[A,B,C](f: (A, B) => C): A => (B => C) =
    (a: A) => ((b: B) => f(a, b))

defined function curry

## EXERCISE 2.4

Implement uncurry, which reverses the transformation of curry. Note that since =>
associates to the right, A => (B => C) can be written as A => B => C.

```scala
def uncurry[A,B,C](f: A => B => C): (A, B) => C
```

In [26]:
def uncurry[A,B,C](f: A => B => C): (A, B) => C = 
    (a: A, b: B) => f(a)(b)

defined function uncurry

In [27]:
val f: (Int, Int) => Int = (x: Int, y: Int) => x + y

f: (Int, Int) => Int = ammonite.$sess.cmd26$Helper$$Lambda$2820/1101631969@194da2

In [28]:
curry(f)

res27: Int => Int => Int = ammonite.$sess.cmd24$Helper$$Lambda$2822/565325139@6d3861d1

In [29]:
val f2 = uncurry(curry(f))

f2: (Int, Int) => Int = ammonite.$sess.cmd25$Helper$$Lambda$2824/952528429@24aa6bc8

In [30]:
f(1,2)
f2(1,2)

res29_0: Int = 3
res29_1: Int = 3

## EXERCISE 2.5

Implement the higher-order function that composes two functions.

```scala
def compose[A,B,C](f: B => C, g: A => B): A => C
```

In [31]:
def compose[A,B,C](f: B => C, g: A => B): A => C =
    (a: A) => f(g(a))

defined function compose

In [32]:
val f = (x: Double) => math.Pi / 2 - x

f: Double => Double = ammonite.$sess.cmd31$Helper$$Lambda$2829/822870283@463cb4b8

In [33]:
val cos = f andThen math.sin // Function1

cos: Double => Double = scala.Function1$$Lambda$720/1423430144@2e4565a1

In [35]:
val cos2 = compose(math.sin, f)

cos2: Double => Double = ammonite.$sess.cmd30$Helper$$Lambda$2835/1998102950@2ab58cb2

In [37]:
cos(4.0/5)
cos2(4.0/5)

res36_0: Double = 0.6967067093471654
res36_1: Double = 0.6967067093471654